In [49]:
"Размерность графа"
def from_id_width(id, width):
    return (id % width, id // width)

"Определяем граф"
def draw_tile(graph, id, style, width):
    r = "."
    if 'number' in style and id in style['number']: r = "%d" % style['number'][id] "стоимость для каждой вершины"
    if 'point_to' in style and style['point_to'].get(id, None) is not None: "направление для каждой вершины"
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = ">"
        if x2 == x1 - 1: r = "<"
        if y2 == y1 + 1: r = "v"
        if y2 == y1 - 1: r = "^"
    if 'start' in style and id == style['start']: r = "O" "начало"
    if 'goal' in style and id == style['goal']: r = "X" "конец"
    if 'path' in style and id in style['path']: r = "*" "путь"
    if id in graph.walls: r = "#" * width "преграды"
    return r

"Рисуем граф"
def draw_grid(graph, width=2, **style):
    for y in range(graph.height):
        for x in range(graph.width):
            print("%%-%ds" % width % draw_tile(graph, (x, y), style, width), end="")
        print()

"Определяем класс сеточных графов"        
class SquareGrid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.walls = []
   
    def in_bounds(self, id):
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id):
        return id not in self.walls
    
    def neighbors(self, id):
        (x, y) = id
        results = [(x+1, y), (x, y-1), (x-1, y), (x, y+1)]
        if (x + y) % 2 == 0: results.reverse() # aesthetics
        results = filter(self.in_bounds, results)
        results = filter(self.passable, results)
        return results

"Класс графов с весами"    
class GridWithWeights(SquareGrid):
    def __init__(self, width, height):
        super().__init__(width, height)
        self.weights = {}
    "определяем цены, здесь у всех вершин 1"
    def cost(self, from_node, to_node):
        return self.weights.get(to_node, 1)


import heapq

"Класс очередей с приоритетами"
class PriorityQueue:
    def __init__(self):
        self.elements = []
    
    def empty(self):
        return len(self.elements) == 0
    
    def put(self, item, priority):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self):
        return heapq.heappop(self.elements)[1]

"Определяем разные эвристики"
def heuristic1(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

def heuristic2(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return max(abs(x1 - x2),abs(y1 - y2))

"Алгоритм А*"
def a_star_search(graph, start, goal, heuristic):
    "счетчик количества пройденных вершин"
    counter = 0
    "очередь с приоритетами"
    frontier = PriorityQueue()
    frontier.put(start, 0)
    "кортежи вершин откуда пришли и цены"
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        
        if current == goal:
            break
      
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(goal, next)
                frontier.put(next, priority)
                came_from[next] = current
                counter += 1
    
    return came_from, cost_so_far, counter

"Построение пути"
def reconstruct_path(came_from, start, goal):
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.append(start) 
    path.reverse() 
    return path

In [50]:
"Задаем произвольный граф"    
diagram4 = GridWithWeights(10, 10)
"Его преграды"
diagram4.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8), (3, 9), (1, 2), (4, 5), (3, 5)]

In [54]:
"Алгоритм с 1ой эвристикой"
came_from, cost_so_far, counter = a_star_search(diagram4, (1, 4), (7, 8), heuristic1)
"Число обработанных вершин"
print(counter)
draw_grid(diagram4, width=3, path=reconstruct_path(came_from, start=(1, 4), goal=(7, 8))[2:-1], start=(1, 4), goal=(7, 8))
    

43
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  ###.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  O  .  .  .  .  .  .  .  .  
.  *  .  ######.  .  .  .  .  
.  *  *  *  *  .  .  .  .  .  
.  #########*  .  .  .  .  .  
.  #########*  *  *  X  .  .  
.  .  .  ###.  .  .  .  .  .  


In [53]:
"Алгоритм со 2ой эвристикой"
came_from, cost_so_far, counter = a_star_search(diagram4, (1, 4), (7, 8), heuristic2)
"Число обработанных вершин"
print(counter)
draw_grid(diagram4, width=3, path=reconstruct_path(came_from, start=(1, 4), goal=(7, 8))[2:-1], start=(1, 4), goal=(7, 8))

53
.  .  .  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  ###.  .  .  .  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
.  O  *  .  .  .  .  .  .  .  
.  .  *  ######.  .  .  .  .  
.  .  *  *  *  *  .  .  .  .  
.  #########.  *  *  .  .  .  
.  #########.  .  *  X  .  .  
.  .  .  ###.  .  .  .  .  .  
